# MODEL DATASET CLEANING

we have already removed the online classes using the code in [`model_db.py`](model_db.py)

but the dataset needs cleaning further due to the fact that some schedule-based class end times due not reflect the real-life end times of the classes

In [153]:
import pandas as pd
import sqlite3
from data_collections.constants import SQLITE_DATABASE_MODEL

In [154]:
TABLE_NAME = "course_info" 

In [155]:
connection = sqlite3.connect(SQLITE_DATABASE_MODEL)
c = connection.cursor()

In [156]:
obs_data = pd.read_sql_query(f'SELECT * FROM {TABLE_NAME}', connection)

In [157]:
data = obs_data.copy()

In [158]:
data.head()

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
0,GÖRSEL İLETİŞİM TASARIMI_GSF102,GÖRSEL İLETİŞİM TASARIMI,GSF102,TEMEL TASARIM II,L-102(GİT ATÖLYE II)[60],SEÇİL KARTOPU,1,Pazartesi,09:00,14:50,41
1,GÖRSEL İLETİŞİM TASARIMI_GSF106,GÖRSEL İLETİŞİM TASARIMI,GSF106,ÇİZİM II,L-101(GİT ATÖLYE I)[60],Öğr.Gör. İSMAİL FURKAN YAVUZ,1,Pazartesi,09:00,14:50,44
2,GÖRSEL İLETİŞİM TASARIMI_BLG102,GÖRSEL İLETİŞİM TASARIMI,BLG102,BİLGİ TEKNOLOJİLERİ KULLANIMI,AZ 13(BİLGİSAYAR LAB.)[80],Öğr.Gör. AHMET AKSOY,1,Salı,09:00,15:50,44
3,İKTİSAT_ENG104,İKTİSAT,ENG104,ACADEMIC ENGLISH II,B382(KONFERANS SALONU)[143],Öğr.Gör. MÜGE AKGEDİK AKOĞLANOĞLU,1,Pazartesi,09:00,16:50,35
4,İKTİSAT_ECON102,İKTİSAT,ECON102,INTRODUCTION TO ECONOMICS II,B338-D1(DERSLİK1)[121],KORAY GÖKSAL,1,Pazartesi,14:00,16:50,91


In [159]:
print(f'there are {len(data.columns)} columns and {len(data)} rows')

there are 11 columns and 623 rows


In [160]:
data.to_csv('no1.csv')
o_data = data.copy()

### STEP 1: structure the data for the model

In [161]:
# list all the columns to drop

print(data.columns)

Index(['faculty_code', 'faculty', 'course_code', 'course_name', 'class',
       'teacher', 'year', 'day', 'start_time', 'end_time', 'student_nums'],
      dtype='object')


In [162]:
to_drop = ['faculty_code', 'faculty', 'course_name', 'class', 'year', 'start_time']

In [163]:
data.drop(labels=to_drop,axis=1, inplace=True)

In [164]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [165]:
import pandas as pd
from datetime import datetime, timedelta

def calculate_bus_stop_counts(df):
    """
    Takes a class schedule DataFrame and returns a new DataFrame with time slots and bus_stop_count.
    Works using minutes-from-midnight as time representation.
    
    Required columns in df:
    - 'day' (e.g., Monday)
    - 'end_time' (string in 'HH:MM' format)
    - 'student_nums' (integer)
    """
    
    # Convert end_time to datetime for parsing, then extract minutes-from-midnight
    df['end_time'] = pd.to_datetime(df['end_time'], format='%H:%M')
    df['end_time_minutes'] = df['end_time'].dt.hour * 60 + df['end_time'].dt.minute
    
    time_slots = []

    for _, row in df.iterrows():
        end_time_min = row['end_time_minutes']
        student_count = row['student_nums']
        day = row['day']
        
        # Define departure distribution
        departures = [
            (0, 0.20),  # 0-5 minutes
            (5, 0.50),  # 5-10 minutes
            (10, 0.30)  # 10-15 minutes
        ]

        for offset, fraction in departures:
            slot_min = end_time_min + offset  # Still in minutes-from-midnight
            time_slots.append({
                'day': day,
                'time_slot_minutes': slot_min,
                'bus_stop_count': int(student_count * fraction)
            })

    # Create DataFrame from list of time slots
    result_df = pd.DataFrame(time_slots)

    # Group by day and time slot to sum overlapping students
    result_df = result_df.groupby(['day', 'time_slot_minutes'], as_index=False).sum()
    #result_df['time_slot'] = result_df['time_slot_minutes'].apply(lambda x: f"{x // 60:02}:{x % 60:02}")


    return result_df


In [166]:
data = calculate_bus_stop_counts(data)
print(data)

          day  time_slot_minutes  bus_stop_count
0        Cuma                650              30
1        Cuma                655              78
2        Cuma                660              46
3        Cuma                710              91
4        Cuma                715             232
..        ...                ...             ...
158  Çarşamba               1075              63
159  Çarşamba               1080              37
160  Çarşamba               1130              36
161  Çarşamba               1135              91
162  Çarşamba               1140              54

[163 rows x 3 columns]


In [167]:
data.to_csv('no2.csv')
e_data = data.copy()

### original data

In [168]:
o_data

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
0,GÖRSEL İLETİŞİM TASARIMI_GSF102,GÖRSEL İLETİŞİM TASARIMI,GSF102,TEMEL TASARIM II,L-102(GİT ATÖLYE II)[60],SEÇİL KARTOPU,1,Pazartesi,09:00,14:50,41
1,GÖRSEL İLETİŞİM TASARIMI_GSF106,GÖRSEL İLETİŞİM TASARIMI,GSF106,ÇİZİM II,L-101(GİT ATÖLYE I)[60],Öğr.Gör. İSMAİL FURKAN YAVUZ,1,Pazartesi,09:00,14:50,44
2,GÖRSEL İLETİŞİM TASARIMI_BLG102,GÖRSEL İLETİŞİM TASARIMI,BLG102,BİLGİ TEKNOLOJİLERİ KULLANIMI,AZ 13(BİLGİSAYAR LAB.)[80],Öğr.Gör. AHMET AKSOY,1,Salı,09:00,15:50,44
3,İKTİSAT_ENG104,İKTİSAT,ENG104,ACADEMIC ENGLISH II,B382(KONFERANS SALONU)[143],Öğr.Gör. MÜGE AKGEDİK AKOĞLANOĞLU,1,Pazartesi,09:00,16:50,35
4,İKTİSAT_ECON102,İKTİSAT,ECON102,INTRODUCTION TO ECONOMICS II,B338-D1(DERSLİK1)[121],KORAY GÖKSAL,1,Pazartesi,14:00,16:50,91
...,...,...,...,...,...,...,...,...,...,...,...
618,FELSEFE_PSİ340,FELSEFE,PSİ340,Öğrenme Psikolojisi,None,None,4,Çarşamba,09.00,11:50,31
619,FELSEFE_FEL408,FELSEFE,FEL408,Siyaset Felsefesi,None,None,4,Çarşamba,09.00,11:50,36
620,FELSEFE_FEL420,FELSEFE,FEL420,Felsefi Danışmanlık,None,None,4,Çarşamba,13.00,14:50,23
621,FELSEFE_FEL424,FELSEFE,FEL424,İletişim Felsefesi,None,None,4,Perşembe,13.00,14:50,19


### data after preprocessing

In [169]:
e_data

,day,time_slot_minutes,bus_stop_count
0,Cuma,650,30
1,Cuma,655,78
2,Cuma,660,46
3,Cuma,710,91
4,Cuma,715,232
...,...,...,...
158,Çarşamba,1075,63
159,Çarşamba,1080,37
160,Çarşamba,1130,36
161,Çarşamba,1135,91


# PROBLEM: 
some times are missing in the final dataset for example according to the dataset there are no classes ending at 3:15 pm on cuma, even though i was there at that time and there were aproximetly 60 students at the bus stop at that time. at 3:20 the number increased to 75, the dataset does not have this entry as well

## REASON:

in the course schedule the class (ITB414) end time is at 3:50 pm, in real life it ended at 3:10.
when i searched for classes that end at 3:50 on cuma it returned the following:

`     time_slot_minutes  bus_stop_count  density  day_encoded`

`12                 950             118        0            0`

and when we searched for 3:55 it returned:

`    time_slot_minutes  bus_stop_count  density  day_encoded`

`13                 955             304        0            0`

which is still pretty far from reality

You're absolutely right to question this — and your reasoning is solid. What you’re experiencing is a **misalignment between planned class schedules and real-world behavior**, which is a **common and tricky challenge** in modeling human-centric systems like this.

---

## 🧠 Problem Breakdown

### ✅ The model's input:
- Based on **scheduled class end times** (e.g., 15:50 or 3:50 PM).
- These are **idealized**, not always real.

### ❌ The real-world situation:
- Students often **leave early**.
- Especially in longer classes, professors might finish ~30–40 minutes earlier.
- This creates **mismatched spikes** in `bus_stop_count` that the model can't explain based on the official class schedule.

---

## 🔥 Why This Is a Big Deal

Since `bus_stop_count` is your ground-truth label, but your **features (class data)** are sometimes shifted in time, the model:
- Misses early spikes in traffic
- Underpredicts for earlier slots (e.g., 15:15 or 15:20)
- Overpredicts for slots where students **should** be leaving (e.g., 15:50), but are already gone

This hurts accuracy and creates unpredictable behavior during inference.

---

## 🛠️ Potential Solutions

### ✅ 1. **Adjust Class End Times (Buffer Window)**
Instead of treating class end time as exact:
- Subtract **a buffer** (e.g., 30 or 40 minutes) from the scheduled `end_time`
- So a class that ends at **15:50** is treated as ending at **15:10 or 15:20**

You can apply this globally or only to specific types of classes (e.g., longer ones).

**How to do this:**
```python
df['adjusted_end_time'] = df['end_time'] - pd.to_timedelta(30, unit='minutes')
```

This gives you a better alignment to real-world behavior.

---

### ✅ 2. **Use Rolling Windows**
Add rolling context to each time slot:
- `students_leaving_within_15min`
- `students_leaving_within_30min`

This captures **"pending exits"** and softens the dependency on exact end time.

---

### ✅ 3. **Weight Classes Based on Duration**
Longer classes are more likely to **end early**, so you can build a heuristic like:

```python
if duration > 4 hours:
    buffer = 40 mins
elif duration > 2 hours:
    buffer = 25 mins
else:
    buffer = 10 mins
```

Apply this buffer to end times dynamically.

---

### ✅ 4. **Learn from the Error Pattern**
If you keep logs of real-world mismatches (like this 15:15 case), you can:
- Build a correction layer
- Train a model to **learn the time-shift pattern**
- Use it to auto-adjust class time mappings

---

## 💡 Best Combo (Realistic + Practical)

Use a hybrid of:

1. **Global buffer** (e.g., -30 mins from end_time)  
2. **Rolling features** (students_leaving_within_30min)  
3. Add a categorical flag like `is_late_afternoon` to help capture behavioral shifts in the day

---

## 🧭 Final Thought

This issue is *exactly* the kind of subtle, real-world messiness that throws off models that rely on **clean, idealized data**. You're on the right track by questioning these assumptions and pushing for more realistic modeling — this is how real ML work actually gets better.

Would you like help implementing the time-adjusted version of the dataset with rolling windows and buffers?

## SOLUTION:

1. find the classes that end at 10, 11, 12, 13, 14, 15, 16, 17 hours and divide them into groups 

In [170]:
o_data.head()

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
0,GÖRSEL İLETİŞİM TASARIMI_GSF102,GÖRSEL İLETİŞİM TASARIMI,GSF102,TEMEL TASARIM II,L-102(GİT ATÖLYE II)[60],SEÇİL KARTOPU,1,Pazartesi,09:00,14:50,41
1,GÖRSEL İLETİŞİM TASARIMI_GSF106,GÖRSEL İLETİŞİM TASARIMI,GSF106,ÇİZİM II,L-101(GİT ATÖLYE I)[60],Öğr.Gör. İSMAİL FURKAN YAVUZ,1,Pazartesi,09:00,14:50,44
2,GÖRSEL İLETİŞİM TASARIMI_BLG102,GÖRSEL İLETİŞİM TASARIMI,BLG102,BİLGİ TEKNOLOJİLERİ KULLANIMI,AZ 13(BİLGİSAYAR LAB.)[80],Öğr.Gör. AHMET AKSOY,1,Salı,09:00,15:50,44
3,İKTİSAT_ENG104,İKTİSAT,ENG104,ACADEMIC ENGLISH II,B382(KONFERANS SALONU)[143],Öğr.Gör. MÜGE AKGEDİK AKOĞLANOĞLU,1,Pazartesi,09:00,16:50,35
4,İKTİSAT_ECON102,İKTİSAT,ECON102,INTRODUCTION TO ECONOMICS II,B338-D1(DERSLİK1)[121],KORAY GÖKSAL,1,Pazartesi,14:00,16:50,91


In [171]:
end_10_before_00 = o_data.loc[o_data['end_time'] == '23:50']
end_00 = o_data.loc[o_data['end_time'] == '00:00']
end_10_before_01 = o_data.loc[o_data['end_time'] == '00:50']
end_01 = o_data.loc[o_data['end_time'] == '01:00']
end_10_before_02 = o_data.loc[o_data['end_time'] == '01:50']
end_02 = o_data.loc[o_data['end_time'] == '02:00']
end_10_before_03 = o_data.loc[o_data['end_time'] == '02:50']
end_03 = o_data.loc[o_data['end_time'] == '03:00']
end_10_before_04 = o_data.loc[o_data['end_time'] == '03:50']
end_04 = o_data.loc[o_data['end_time'] == '04:00']
end_10_before_05 = o_data.loc[o_data['end_time'] == '04:50']
end_05 = o_data.loc[o_data['end_time'] == '05:00']
end_10_before_06 = o_data.loc[o_data['end_time'] == '05:50']
end_06 = o_data.loc[o_data['end_time'] == '06:00']
end_10_before_07 = o_data.loc[o_data['end_time'] == '06:50']
end_07 = o_data.loc[o_data['end_time'] == '07:00']
end_10_before_08 = o_data.loc[o_data['end_time'] == '07:50']
end_08 = o_data.loc[o_data['end_time'] == '08:00']
end_10_before_09 = o_data.loc[o_data['end_time'] == '08:50']
end_09 = o_data.loc[o_data['end_time'] == '09:00']
end_10_before_10 = o_data.loc[o_data['end_time'] == '09:50']
end_10 = o_data.loc[o_data['end_time'] == '10:00']
end_10_before_11 = o_data.loc[o_data['end_time'] == '10:50']
end_11 = o_data.loc[o_data['end_time'] == '11:00']
end_10_before_12 = o_data.loc[o_data['end_time'] == '11:50']
end_12 = o_data.loc[o_data['end_time'] == '12:00']
end_10_before_13 = o_data.loc[o_data['end_time'] == '12:50']
end_13 = o_data.loc[o_data['end_time'] == '13:00']
end_10_before_14 = o_data.loc[o_data['end_time'] == '13:50']
end_14 = o_data.loc[o_data['end_time'] == '14:00']
end_10_before_15 = o_data.loc[o_data['end_time'] == '14:50']
end_15 = o_data.loc[o_data['end_time'] == '15:00']
end_10_before_16 = o_data.loc[o_data['end_time'] == '15:50']
end_16 = o_data.loc[o_data['end_time'] == '16:00']
end_10_before_17 = o_data.loc[o_data['end_time'] == '16:50']
end_17 = o_data.loc[o_data['end_time'] == '17:00']
end_10_before_18 = o_data.loc[o_data['end_time'] == '17:50']
end_18 = o_data.loc[o_data['end_time'] == '18:00']
end_10_before_19 = o_data.loc[o_data['end_time'] == '18:50']
end_19 = o_data.loc[o_data['end_time'] == '19:00']
end_10_before_20 = o_data.loc[o_data['end_time'] == '19:50']
end_20 = o_data.loc[o_data['end_time'] == '20:00']
end_10_before_21 = o_data.loc[o_data['end_time'] == '20:50']
end_21 = o_data.loc[o_data['end_time'] == '21:00']
end_10_before_22 = o_data.loc[o_data['end_time'] == '21:50']
end_22 = o_data.loc[o_data['end_time'] == '22:00']
end_10_before_23 = o_data.loc[o_data['end_time'] == '22:50']
end_23 = o_data.loc[o_data['end_time'] == '23:00']

In [172]:
end_times = [
    {'end_time': '23:50', 'count': len(end_10_before_00)},
    {'end_time': '00:00', 'count': len(end_00)},
    {'end_time': '00:50', 'count': len(end_10_before_01)},
    {'end_time': '01:00', 'count': len(end_01)},
    {'end_time': '01:50', 'count': len(end_10_before_02)},
    {'end_time': '02:00', 'count': len(end_02)},
    {'end_time': '02:50', 'count': len(end_10_before_03)},
    {'end_time': '03:00', 'count': len(end_03)},
    {'end_time': '03:50', 'count': len(end_10_before_04)},
    {'end_time': '04:00', 'count': len(end_04)},
    {'end_time': '04:50', 'count': len(end_10_before_05)},
    {'end_time': '05:00', 'count': len(end_05)},
    {'end_time': '05:50', 'count': len(end_10_before_06)},
    {'end_time': '06:00', 'count': len(end_06)},
    {'end_time': '06:50', 'count': len(end_10_before_07)},
    {'end_time': '07:00', 'count': len(end_07)},
    {'end_time': '07:50', 'count': len(end_10_before_08)},
    {'end_time': '08:00', 'count': len(end_08)},
    {'end_time': '08:50', 'count': len(end_10_before_09)},
    {'end_time': '09:00', 'count': len(end_09)},
    {'end_time': '09:50', 'count': len(end_10_before_10)},
    {'end_time': '10:00', 'count': len(end_10)},
    {'end_time': '10:50', 'count': len(end_10_before_11)},
    {'end_time': '11:00', 'count': len(end_11)},
    {'end_time': '11:50', 'count': len(end_10_before_12)},
    {'end_time': '12:00', 'count': len(end_12)},
    {'end_time': '12:50', 'count': len(end_10_before_13)},
    {'end_time': '13:00', 'count': len(end_13)},
    {'end_time': '13:50', 'count': len(end_10_before_14)},
    {'end_time': '14:00', 'count': len(end_14)},
    {'end_time': '14:50', 'count': len(end_10_before_15)},
    {'end_time': '15:00', 'count': len(end_15)},
    {'end_time': '15:50', 'count': len(end_10_before_16)},
    {'end_time': '16:00', 'count': len(end_16)},
    {'end_time': '16:50', 'count': len(end_10_before_17)},
    {'end_time': '17:00', 'count': len(end_17)},
    {'end_time': '17:50', 'count': len(end_10_before_18)},
    {'end_time': '18:00', 'count': len(end_18)},
    {'end_time': '18:50', 'count': len(end_10_before_19)},
    {'end_time': '19:00', 'count': len(end_19)},
    {'end_time': '19:50', 'count': len(end_10_before_20)},
    {'end_time': '20:00', 'count': len(end_20)},
    {'end_time': '20:50', 'count': len(end_10_before_21)},
    {'end_time': '21:00', 'count': len(end_21)},
    {'end_time': '21:50', 'count': len(end_10_before_22)},
    {'end_time': '22:00', 'count': len(end_22)},
    {'end_time': '22:50', 'count': len(end_10_before_23)},
    {'end_time': '23:00', 'count': len(end_23)},

]

In [173]:
end_times_df = pd.DataFrame(end_times)

In [174]:
end_times_df

,end_time,count
0,23:50,0
1,00:00,0
2,00:50,0
3,01:00,0
4,01:50,0
5,02:00,0
6,02:50,0
7,03:00,0
8,03:50,0
9,04:00,0


In [175]:
classes_end_times = end_times_df.loc[end_times_df['count'] > 0]

In [176]:
classes_end_times

,end_time,count
20,09:50,14
21,10:00,3
22,10:50,27
23,11:00,1
24,11:50,174
25,12:00,6
26,12:50,26
27,13:00,1
28,13:50,29
29,14:00,1


In [177]:
len(classes_end_times)

18

as we can see from the `classes_end_times` dataframe, there are only 18 end_times with claases ending at that time, with 3 times dominating the dataset:

| end_time | classes |
| -------- | ------- |
| 11:50	  | 174    |
| 15:50	 | 134     |
| 16:50	    | 116    |

then we have `14:50` with `70` classes ending at that time

next, we will investigate each of the end_times to remove potential outliers


### NOTE:
some classes that have 0 students may need to be dropped, we placed 0 as a place holder because we could not find the actual number of students taking the class and they are probably influencing model performance:

alternative option is to take the average of similar classes or if the class is mandatory place the value of a similar mandatory class in the same year

In [178]:
all_end_time_dfs = [
    end_10_before_00, end_00, end_10_before_01, end_01,
    end_10_before_02, end_02, end_10_before_03, end_03,
    end_10_before_04, end_04,end_10_before_05, end_05,
    end_10_before_06, end_06, end_10_before_07, end_07,
    end_10_before_08, end_08, end_10_before_09, end_09,
    end_10_before_10, end_10, end_10_before_11, end_11,
    end_10_before_12, end_12, end_10_before_13, end_13,
    end_10_before_14, end_14, end_10_before_15, end_15,
    end_10_before_16, end_16, end_10_before_17, end_17,
    end_10_before_18, end_18, end_10_before_19, end_19,
    end_10_before_20, end_20, end_10_before_21, end_21,
    end_10_before_22, end_22, end_10_before_23, end_23
]

In [179]:
end_times_match_classes = []
for df in all_end_time_dfs:

    if len(df) > 0:
        end_times_match_classes.append(df)


In [180]:
print(len(end_times_match_classes))

18


change of approach:
all end times with under 20 classes will be checked

In [181]:
e = []

for df in end_times_match_classes:

    if len(df) >= 20:
        pass
    else:
        e.append(df)

`SOS404	BİTİRME PROJESİ II` looks like it should not be there

In [182]:
len(e)

11

In [183]:
e[0]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
50,TÜRK DİLİ VE EDEBİYATI_TDE244,TÜRK DİLİ VE EDEBİYATI,TDE244,SES EĞİTİMİ VE ŞİİR OKUMA,A 367(YUSUF HAS HÂCİP DERSLİĞİ)[90],SALİM PİLAV,2,Salı,08:00,09:50,21
180,TÜRK DİLİ VE EDEBİYATI_TDE344,TÜRK DİLİ VE EDEBİYATI,TDE344,ESKİ TÜRK EDEBİYATINDA ARAŞTIRMA YÖNTEMLERİ II,A 369 (AHMET HAMDİ TANPINAR DERSLİĞi)[80],MUSTAFA YASİN BAŞÇETİN,3,Çarşamba,08:00,09:50,0
240,SOSYOLOJİ_SOS404,SOSYOLOJİ,SOS404,BİTİRME PROJESİ II,A 280(SABRİ ÜLGENER DERSLİĞİ)[70],HALİME ÜNAL REŞİTOĞLU,4,Perşembe,08:00,09:50,8
341,HEMŞİRELİK_HEM308,HEMŞİRELİK,HEM308,YOĞUN BAKIM HEMŞİRELİĞİ,None,None,3,Salı,08.00,09:50,91
345,HEMŞİRELİK_HEM416,HEMŞİRELİK,HEM416,ÇOCUK İHMALİ VE İSTİSMARI,None,None,4,Pazartesi,08.00,09:50,44
392,BESLENME VE_SBF104,BESLENME VE,SBF104,SAĞLIK İLETİŞİMİ,None,None,1,Perşembe,08.00,09:50,208
399,BESLENME VE_BED218,BESLENME VE,BED218,DÜNYA MUTFAKLARI,None,None,2,Çarşamba,08.00,09:50,79
402,BESLENME VE_BED320,BESLENME VE,BED320,ENTERAL VE PARENTERAL BESLENME,None,None,3,Pazartesi,08.00,09:50,66
438,FİZYOTERAPİ_FTR106,FİZYOTERAPİ,FTR106,Beslenme,None,None,1,Salı,08.00,09:50,94
441,FİZYOTERAPİ_FTR108,FİZYOTERAPİ,FTR108,Biyokimya,None,None,1,Perşembe,08.00,09:50,116


bitirme projesi to drop

In [184]:
e[1]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
495,SAĞLIK YÖNE_SY112,SAĞLIK YÖNE,SY112,YÖNETİM BİLİMLERİNE GİRİŞ,None,None,1,Perşembe,8.00,10:00,100
499,SAĞLIK YÖNE_SY212,SAĞLIK YÖNE,SY212,SAĞLIK TURİZMİ,None,None,2,Salı,8.00,10:00,61
505,SAĞLIK YÖNE_SY312,SAĞLIK YÖNE,SY312,SAĞLIK HİZMETLERİNDE HALKLA İLİŞKİLER VE İLETİŞİM,None,None,3,Pazartesi,8.00,10:00,58


In [185]:
e[2]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
497,SAĞLIK YÖNE_SY210,SAĞLIK YÖNE,SY210,SAĞLIKTA YENİLİKÇİLİK VE GİRİŞİMCİLİK,None,None,2,Pazartesi,9:00,11:00,59


In [186]:
e[3]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
494,SAĞLIK YÖNE_SY114,SAĞLIK YÖNE,SY114,İŞLETME MATEMATİĞİ,None,None,1,Çarşamba,10.00,12:00,98
498,SAĞLIK YÖNE_SY211,SAĞLIK YÖNE,SY211,ZAMAN YÖNETİMİ,None,None,2,Pazartesi,10.00,12:00,17
500,SAĞLIK YÖNE_SY222,SAĞLIK YÖNE,SY222,HASTA HAKLARI,None,None,2,Salı,10:00,12:00,46
504,SAĞLIK YÖNE_SY202,SAĞLIK YÖNE,SY202,İDARE HUKUKU,None,None,2,Perşembe,10.00,12:00,87
507,SAĞLIK YÖNE_SY304,SAĞLIK YÖNE,SY304,SAĞLIK HUKUKU,None,None,3,Salı,10.00,12:00,60
509,SAĞLIK YÖNE_SY306,SAĞLIK YÖNE,SY306,SAĞLIK HİZMETLERİNDE PERFORMANS YÖNETİMİ,None,None,3,Çarşamba,10.00,12:00,67


In [187]:
e[4]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
511,SAĞLIK YÖNE_SY406,SAĞLIK YÖNE,SY406,BİTİRME PROJESİ,None,None,4,Pazartesi,11.00,13:00,55


this may need to be dropped, because the only class seems to be a final project

In [188]:
e[5]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
492,SAĞLIK YÖNE_SY118,SAĞLIK YÖNE,SY118,HALK SAĞLIĞI,None,None,1,Salı,11.00,14:00,95


In [189]:
e[6]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
496,SAĞLIK YÖNE_SY104,SAĞLIK YÖNE,SY104,İKTİSAT II,None,None,1,Perşembe,13.00,15:00,110
502,SAĞLIK YÖNE_SY220,SAĞLIK YÖNE,SY220,SAĞLIK KURUMLARINDA MALİYET MUHASEBESİ,None,None,2,Salı,13:00,15:00,59
503,SAĞLIK YÖNE_SY204,SAĞLIK YÖNE,SY204,SAĞLIK POLİTİKASI,None,None,2,Çarşamba,13:00,15:00,70
506,SAĞLIK YÖNE_SY316,SAĞLIK YÖNE,SY316,SAĞLIK KURUMLARINDA ÜRETİM YÖNETİMİ,None,None,3,Salı,13.00,15:00,65
510,SAĞLIK YÖNE_SY308,SAĞLIK YÖNE,SY308,SAĞLIK HİZMETLERİNDE KALİTE YÖNETİMİ,None,None,3,Perşembe,13.00,15:00,60


In [190]:
e[7]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
493,SAĞLIK YÖNE_SY106,SAĞLIK YÖNE,SY106,SAĞLIK HİZMETLERİNDE ETİK,None,None,1,Salı,15.00,17:00,83
501,SAĞLIK YÖNE_SY206,SAĞLIK YÖNE,SY206,SAĞLIK KURUMLARINDA ÖRGÜTSEL DAVRANIŞ,None,None,2,Salı,15.00,17:00,60
508,SAĞLIK YÖNE_SY310,SAĞLIK YÖNE,SY310,HASTA GÜVENLİĞİ,None,None,3,Salı,15:00,17:00,62


In [191]:
e[8]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
88,MİMARLIK(İNG)_ARCH216,MİMARLIK(İNG),ARCH216,STRUCTURAL BUILDING SYSTEMS AND COMPONENTS,L-303(DERSLİK)[120],EKREM BAHADIR ÇALIŞKAN,2,Pazartesi,13:00,17:50,102
106,PSİKOLOJİ(İNG)_PSY202,PSİKOLOJİ(İNG),PSY202,SOCIAL PSYCHOLOGY II,AMFİ A 172(AMFİ A 172 )[130],CEMŞAFAK ÇUKUR,2,Pazartesi,15:00,17:50,114
149,MİMARLIK(İNG)_ARCHE322,MİMARLIK(İNG),ARCHE322,HISTORICAL TURKISH CITIES,L-303(DERSLİK)[120],KÜBRA YILDIRIM ÖZCAN,3,Perşembe,15:00,17:50,73
161,PSİKOLOJİ(İNG)_PSY302,PSİKOLOJİ(İNG),PSY302,PERSONALITY THEORIES II,A 247(DUYGU DERSLİĞİ)[60],ELİF EMİR ÖKSÜZ,3,Çarşamba,13:00,17:50,85
219,MİMARLIK(İNG)_ARCH402,MİMARLIK(İNG),ARCH402,STUDIO VIII (FINAL PROJECT),L215(ATÖLYE V)[80],AHMET EMRE DİNÇER,4,Perşembe,09:00,17:50,8
567,TARİH_TAR 315,TARİH,TAR 315,Osmanlı Tarihi Kaynakları,None,None,3,Salı,16.00,17:50,0
585,FELSEFE_FEL104,FELSEFE,FEL104,Felsefi Kavramlar ve Terimler,None,None,1,Pazartesi,15.00,17:50,0
597,FELSEFE_FEL204,FELSEFE,FEL204,Ahlak Felsefesi,None,None,2,Çarşamba,15.00,17:50,42
615,FELSEFE_FEL404,FELSEFE,FEL404,Bitirme Tezi-II,None,None,4,Pazartesi,16.00,17:50,5


In [192]:
e[9]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
91,MİMARLIK(İNG)_ARCHE233,MİMARLIK(İNG),ARCHE233,INTRODUCTION TO SOCIOLOGY,L-301(MAKET ATÖLYESİ)[65],RÜYA ARDIÇOĞLU,2,Salı,16:00,18:50,67
94,MİMARLIK(İNG)_ARCH234,MİMARLIK(İNG),ARCH234,THEORIES OF URBAN DESIGN,L-318/319(ATÖLYE IV)[140],KÜBRA YILDIRIM ÖZCAN,2,Çarşamba,16:00,18:50,183
145,MİMARLIK(İNG)_ARCH334,MİMARLIK(İNG),ARCH334,CONSERVATION AND RESTORATION OF BUILT HERITAGE,L-302(ATÖLYE I)[65],FİLİZ KARAKUŞ,3,Salı,16:00,18:50,17


these start at 4 and end at 6:50, so they are probably outside of campus, but we need to make sure

In [193]:
e[10]

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
220,MİMARLIK(İNG)_ARCH404,MİMARLIK(İNG),ARCH404,RESEARCH THESIS,L215(ATÖLYE V)[80],AHMET EMRE DİNÇER,4,Perşembe,18:00,19:50,11


this is probably a graduation projrect

now we drop the courses

In [194]:
# courses_to_drop list as you have
courses_to_drop = ['MİMARLIK(İNG)_ARCH404', 'MİMARLIK(İNG)_ARCHE233',
                   'MİMARLIK(İNG)_ARCH234', 'MİMARLIK(İNG)_ARCH334',
                   'SAĞLIK YÖNE_SY406', 'SOSYOLOJİ_SOS404']

# Assuming new_o_data is a copy of o_data
new_o_data = o_data.copy()

# Drop rows where 'faculty_code' is in courses_to_drop
new_o_data = new_o_data[~new_o_data['faculty_code'].isin(courses_to_drop)].copy()

# Now new_o_data does NOT contain the courses in courses_to_drop
print(new_o_data)


                        faculty_code                   faculty course_code  \
0    GÖRSEL İLETİŞİM TASARIMI_GSF102  GÖRSEL İLETİŞİM TASARIMI      GSF102   
1    GÖRSEL İLETİŞİM TASARIMI_GSF106  GÖRSEL İLETİŞİM TASARIMI      GSF106   
2    GÖRSEL İLETİŞİM TASARIMI_BLG102  GÖRSEL İLETİŞİM TASARIMI      BLG102   
3                     İKTİSAT_ENG104                   İKTİSAT      ENG104   
4                    İKTİSAT_ECON102                   İKTİSAT     ECON102   
..                               ...                       ...         ...   
618                   FELSEFE_PSİ340                   FELSEFE      PSİ340   
619                   FELSEFE_FEL408                   FELSEFE      FEL408   
620                  FELSEFE_FEL420                    FELSEFE     FEL420    
621                   FELSEFE_FEL424                   FELSEFE      FEL424   
622                   FELSEFE_FEL422                   FELSEFE      FEL422   

                       course_name                        class

drop the zero student nums for now

In [195]:
new_o_data = new_o_data.loc[new_o_data['student_nums'] != 0]

In [196]:
new_o_data

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
0,GÖRSEL İLETİŞİM TASARIMI_GSF102,GÖRSEL İLETİŞİM TASARIMI,GSF102,TEMEL TASARIM II,L-102(GİT ATÖLYE II)[60],SEÇİL KARTOPU,1,Pazartesi,09:00,14:50,41
1,GÖRSEL İLETİŞİM TASARIMI_GSF106,GÖRSEL İLETİŞİM TASARIMI,GSF106,ÇİZİM II,L-101(GİT ATÖLYE I)[60],Öğr.Gör. İSMAİL FURKAN YAVUZ,1,Pazartesi,09:00,14:50,44
2,GÖRSEL İLETİŞİM TASARIMI_BLG102,GÖRSEL İLETİŞİM TASARIMI,BLG102,BİLGİ TEKNOLOJİLERİ KULLANIMI,AZ 13(BİLGİSAYAR LAB.)[80],Öğr.Gör. AHMET AKSOY,1,Salı,09:00,15:50,44
3,İKTİSAT_ENG104,İKTİSAT,ENG104,ACADEMIC ENGLISH II,B382(KONFERANS SALONU)[143],Öğr.Gör. MÜGE AKGEDİK AKOĞLANOĞLU,1,Pazartesi,09:00,16:50,35
4,İKTİSAT_ECON102,İKTİSAT,ECON102,INTRODUCTION TO ECONOMICS II,B338-D1(DERSLİK1)[121],KORAY GÖKSAL,1,Pazartesi,14:00,16:50,91
...,...,...,...,...,...,...,...,...,...,...,...
618,FELSEFE_PSİ340,FELSEFE,PSİ340,Öğrenme Psikolojisi,None,None,4,Çarşamba,09.00,11:50,31
619,FELSEFE_FEL408,FELSEFE,FEL408,Siyaset Felsefesi,None,None,4,Çarşamba,09.00,11:50,36
620,FELSEFE_FEL420,FELSEFE,FEL420,Felsefi Danışmanlık,None,None,4,Çarşamba,13.00,14:50,23
621,FELSEFE_FEL424,FELSEFE,FEL424,İletişim Felsefesi,None,None,4,Perşembe,13.00,14:50,19


now that we got rid of unwanted classes we can divide them by day

In [197]:
classes_end_times

,end_time,count
20,09:50,14
21,10:00,3
22,10:50,27
23,11:00,1
24,11:50,174
25,12:00,6
26,12:50,26
27,13:00,1
28,13:50,29
29,14:00,1


# to copy

days_syntax = [{

    'Cuma': len(),
    'Salı': len(),
    'Çarşamba': len(),
    'Pazartesi': len(),
    'Perşembe': len()
}]

columns = ['day', 'class count']

In [198]:
columns_days = ['day', 'class count']

In [199]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_10_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '09:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_10_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '09:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_10_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '09:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_10_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '09:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_10_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '09:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_10_days = [{
    'Cuma': len(end_10_before_10_cuma),
    'Salı': len(end_10_before_10_sali),
    'Çarşamba': len(end_10_before_10_carsamba),
    'Pazartesi': len(end_10_before_10_pazartesi),
    'Perşembe': len(end_10_before_10_persembe)
}]
print(end_10_before_10_days)


[{'Cuma': 0, 'Salı': 4, 'Çarşamba': 1, 'Pazartesi': 4, 'Perşembe': 2}]


In [200]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:00') & (new_o_data['day_clean'] == 'Salı')
]
end_10_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_days = [{
    'Cuma': len(end_10_cuma),
    'Salı': len(end_10_sali),
    'Çarşamba': len(end_10_carsamba),
    'Pazartesi': len(end_10_pazartesi),
    'Perşembe': len(end_10_persembe)
}]
print(end_10_days)


[{'Cuma': 0, 'Salı': 1, 'Çarşamba': 0, 'Pazartesi': 1, 'Perşembe': 1}]


In [201]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_11_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_11_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_11_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_11_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_11_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '10:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_11_days = [{
    'Cuma': len(end_10_before_11_cuma),
    'Salı': len(end_10_before_11_sali),
    'Çarşamba': len(end_10_before_11_carsamba),
    'Pazartesi': len(end_10_before_11_pazartesi),
    'Perşembe': len(end_10_before_11_persembe)
}]
print(end_10_before_11_days)

[{'Cuma': 3, 'Salı': 5, 'Çarşamba': 5, 'Pazartesi': 8, 'Perşembe': 3}]


In [202]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_11_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_11_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:00') & (new_o_data['day_clean'] == 'Salı')
]
end_11_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_11_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_11_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_11_days = [{
    'Cuma': len(end_11_cuma),
    'Salı': len(end_11_sali),
    'Çarşamba': len(end_11_carsamba),
    'Pazartesi': len(end_11_pazartesi),
    'Perşembe': len(end_11_persembe)
}]
print(end_11_days)

[{'Cuma': 0, 'Salı': 0, 'Çarşamba': 0, 'Pazartesi': 1, 'Perşembe': 0}]


In [203]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_12_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_12_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_12_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_12_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_12_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '11:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_12_days = [{
    'Cuma': len(end_10_before_12_cuma),
    'Salı': len(end_10_before_12_sali),
    'Çarşamba': len(end_10_before_12_carsamba),
    'Pazartesi': len(end_10_before_12_pazartesi),
    'Perşembe': len(end_10_before_12_persembe)
}]
print(end_10_before_12_days)

[{'Cuma': 8, 'Salı': 43, 'Çarşamba': 48, 'Pazartesi': 40, 'Perşembe': 28}]


In [204]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_12_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_12_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:00') & (new_o_data['day_clean'] == 'Salı')
]
end_12_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_12_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_12_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_12_days = [{
    'Cuma': len(end_12_cuma),
    'Salı': len(end_12_sali),
    'Çarşamba': len(end_12_carsamba),
    'Pazartesi': len(end_12_pazartesi),
    'Perşembe': len(end_12_persembe)
}]
print(end_12_days)

[{'Cuma': 0, 'Salı': 2, 'Çarşamba': 2, 'Pazartesi': 1, 'Perşembe': 1}]


In [205]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_13_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_13_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_13_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_13_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_13_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '12:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_13_days = [{
    'Cuma': len(end_10_before_13_cuma),
    'Salı': len(end_10_before_13_sali),
    'Çarşamba': len(end_10_before_13_carsamba),
    'Pazartesi': len(end_10_before_13_pazartesi),
    'Perşembe': len(end_10_before_13_persembe)
}]
print(end_10_before_13_days)

[{'Cuma': 0, 'Salı': 7, 'Çarşamba': 5, 'Pazartesi': 9, 'Perşembe': 4}]


In [206]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_13_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_13_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:00') & (new_o_data['day_clean'] == 'Salı')
]
end_13_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_13_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_13_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_13_days = [{
    'Cuma': len(end_13_cuma),
    'Salı': len(end_13_sali),
    'Çarşamba': len(end_13_carsamba),
    'Pazartesi': len(end_13_pazartesi),
    'Perşembe': len(end_13_persembe)
}]
print(end_13_days)

[{'Cuma': 0, 'Salı': 0, 'Çarşamba': 0, 'Pazartesi': 0, 'Perşembe': 0}]


In [207]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_14_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_14_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_14_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_14_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_14_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '13:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_14_days = [{
    'Cuma': len(end_10_before_14_cuma),
    'Salı': len(end_10_before_14_sali),
    'Çarşamba': len(end_10_before_14_carsamba),
    'Pazartesi': len(end_10_before_14_pazartesi),
    'Perşembe': len(end_10_before_14_persembe)
}]
print(end_10_before_14_days)

[{'Cuma': 1, 'Salı': 6, 'Çarşamba': 5, 'Pazartesi': 7, 'Perşembe': 6}]


In [208]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_14_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_14_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:00') & (new_o_data['day_clean'] == 'Salı')
]
end_14_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_14_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_14_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_14_days = [{
    'Cuma': len(end_14_cuma),
    'Salı': len(end_14_sali),
    'Çarşamba': len(end_14_carsamba),
    'Pazartesi': len(end_14_pazartesi),
    'Perşembe': len(end_14_persembe)
}]
print(end_14_days)

[{'Cuma': 0, 'Salı': 1, 'Çarşamba': 0, 'Pazartesi': 0, 'Perşembe': 0}]


In [209]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_15_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_15_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_15_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_15_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_15_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '14:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_15_days = [{
    'Cuma': len(end_10_before_15_cuma),
    'Salı': len(end_10_before_15_sali),
    'Çarşamba': len(end_10_before_15_carsamba),
    'Pazartesi': len(end_10_before_15_pazartesi),
    'Perşembe': len(end_10_before_15_persembe)
}]
print(end_10_before_15_days)

[{'Cuma': 5, 'Salı': 14, 'Çarşamba': 18, 'Pazartesi': 19, 'Perşembe': 9}]


In [210]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_15_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_15_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:00') & (new_o_data['day_clean'] == 'Salı')
]
end_15_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_15_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_15_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_15_days = [{
    'Cuma': len(end_15_cuma),
    'Salı': len(end_15_sali),
    'Çarşamba': len(end_15_carsamba),
    'Pazartesi': len(end_15_pazartesi),
    'Perşembe': len(end_15_persembe)
}]
print(end_15_days)

[{'Cuma': 0, 'Salı': 2, 'Çarşamba': 1, 'Pazartesi': 0, 'Perşembe': 2}]


In [211]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_16_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_16_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_16_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_16_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_16_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '15:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_16_days = [{
    'Cuma': len(end_10_before_16_cuma),
    'Salı': len(end_10_before_16_sali),
    'Çarşamba': len(end_10_before_16_carsamba),
    'Pazartesi': len(end_10_before_16_pazartesi),
    'Perşembe': len(end_10_before_16_persembe)
}]
print(end_10_before_16_days)

[{'Cuma': 10, 'Salı': 38, 'Çarşamba': 29, 'Pazartesi': 29, 'Perşembe': 23}]


In [212]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_17_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '16:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_17_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '16:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_17_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '16:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_17_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '16:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_17_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '16:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_17_days = [{
    'Cuma': len(end_10_before_17_cuma),
    'Salı': len(end_10_before_17_sali),
    'Çarşamba': len(end_10_before_17_carsamba),
    'Pazartesi': len(end_10_before_17_pazartesi),
    'Perşembe': len(end_10_before_17_persembe)
}]
print(end_10_before_17_days)

[{'Cuma': 5, 'Salı': 33, 'Çarşamba': 17, 'Pazartesi': 33, 'Perşembe': 21}]


In [213]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_17_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:00') & (new_o_data['day_clean'] == 'Cuma')
]
end_17_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:00') & (new_o_data['day_clean'] == 'Salı')
]
end_17_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:00') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_17_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:00') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_17_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:00') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_17_days = [{
    'Cuma': len(end_17_cuma),
    'Salı': len(end_17_sali),
    'Çarşamba': len(end_17_carsamba),
    'Pazartesi': len(end_17_pazartesi),
    'Perşembe': len(end_17_persembe)
}]
print(end_17_days)

[{'Cuma': 0, 'Salı': 3, 'Çarşamba': 0, 'Pazartesi': 0, 'Perşembe': 0}]


In [214]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_18_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_18_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_18_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_18_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_18_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '17:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_18_days = [{
    'Cuma': len(end_10_before_18_cuma),
    'Salı': len(end_10_before_18_sali),
    'Çarşamba': len(end_10_before_18_carsamba),
    'Pazartesi': len(end_10_before_18_pazartesi),
    'Perşembe': len(end_10_before_18_persembe)
}]
print(end_10_before_18_days)

[{'Cuma': 0, 'Salı': 0, 'Çarşamba': 2, 'Pazartesi': 3, 'Perşembe': 2}]


In [215]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_19_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '18:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_19_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '18:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_19_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '18:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_19_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '18:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_19_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '18:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_19_days = [{
    'Cuma': len(end_10_before_19_cuma),
    'Salı': len(end_10_before_19_sali),
    'Çarşamba': len(end_10_before_19_carsamba),
    'Pazartesi': len(end_10_before_19_pazartesi),
    'Perşembe': len(end_10_before_19_persembe)
}]
print(end_10_before_19_days)

[{'Cuma': 0, 'Salı': 0, 'Çarşamba': 0, 'Pazartesi': 0, 'Perşembe': 0}]


In [216]:
# Normalize 'end_time' and 'day' columns to ensure consistent matching
new_o_data['end_time_clean'] = new_o_data['end_time'].str.strip()
new_o_data['day_clean'] = new_o_data['day'].str.strip().str.capitalize()

# Filter for each specified day with '09:50' end_time
end_10_before_20_cuma = new_o_data.loc[
    (new_o_data['end_time_clean'] == '19:50') & (new_o_data['day_clean'] == 'Cuma')
]
end_10_before_20_sali = new_o_data.loc[
    (new_o_data['end_time_clean'] == '19:50') & (new_o_data['day_clean'] == 'Salı')
]
end_10_before_20_carsamba = new_o_data.loc[
    (new_o_data['end_time_clean'] == '19:50') & (new_o_data['day_clean'] == 'Çarşamba')
]
end_10_before_20_pazartesi = new_o_data.loc[
    (new_o_data['end_time_clean'] == '19:50') & (new_o_data['day_clean'] == 'Pazartesi')
]
end_10_before_20_persembe = new_o_data.loc[
    (new_o_data['end_time_clean'] == '19:50') & (new_o_data['day_clean'] == 'Perşembe')
]

# Optional: print counts to verify filtering
end_10_before_20_days = [{
    'Cuma': len(end_10_before_20_cuma),
    'Salı': len(end_10_before_20_sali),
    'Çarşamba': len(end_10_before_20_carsamba),
    'Pazartesi': len(end_10_before_20_pazartesi),
    'Perşembe': len(end_10_before_20_persembe)
}]
print(end_10_before_20_days)

[{'Cuma': 0, 'Salı': 0, 'Çarşamba': 0, 'Pazartesi': 0, 'Perşembe': 0}]


In [217]:
ccc = ['day', '09:50', '10:00', '10:50', '11:00', '11:50', '12:00', '12:50', '13:00',
       '13:50', '14:00', '14:50', '15:00', '15:50', '16:50', '17:00', '17:50', '18:50',
        '19:50']

In [218]:
days = ['Cuma', 'Salı', 'Çarşamba', 'Pazartesi', 'Perşembe']

In [219]:
time_list = [end_10_before_10_days, end_10_days, end_10_before_11_days, end_11_days,
             end_10_before_12_days, end_12_days, end_10_before_13_days, end_13_days,
             end_10_before_14_days, end_14_days, end_10_before_15_days, end_15_days,
             end_10_before_16_days, end_10_before_17_days, end_17_days,
             end_10_before_18_days, end_10_before_19_days, end_10_before_20_days]

In [220]:
# Initialize an empty DataFrame with 'day' as the first column
adata = {'day': days}

# Loop through each time slot and fill data from the corresponding dict
for time_str, time_data in zip(ccc[1:], time_list):
    # time_data is a list with a single dict inside
    time_dict = time_data[0]
    # Extract the count for each day in the correct order
    adata[time_str] = [time_dict.get(day, 0) for day in days]

# Build the final DataFrame
final_df = pd.DataFrame(adata)

# Optional: sort by day or reorder columns
# final_df = final_df.set_index('day')  # if you want 'day' as index

print(final_df)


         day  09:50  10:00  10:50  11:00  11:50  12:00  12:50  13:00  13:50  \
0       Cuma      0      0      3      0      8      0      0      0      1   
1       Salı      4      1      5      0     43      2      7      0      6   
2   Çarşamba      1      0      5      0     48      2      5      0      5   
3  Pazartesi      4      1      8      1     40      1      9      0      7   
4   Perşembe      2      1      3      0     28      1      4      0      6   

   14:00  14:50  15:00  15:50  16:50  17:00  17:50  18:50  19:50  
0      0      5      0     10      5      0      0      0      0  
1      1     14      2     38     33      3      0      0      0  
2      0     18      1     29     17      0      2      0      0  
3      0     19      0     29     33      0      3      0      0  
4      0      9      2     23     21      0      2      0      0  


In [221]:
final_df

,day,09:50,10:00,10:50,11:00,11:50,12:00,12:50,13:00,13:50,14:00,14:50,15:00,15:50,16:50,17:00,17:50,18:50,19:50
0,Cuma,0,0,3,0,8,0,0,0,1,0,5,0,10,5,0,0,0,0
1,Salı,4,1,5,0,43,2,7,0,6,1,14,2,38,33,3,0,0,0
2,Çarşamba,1,0,5,0,48,2,5,0,5,0,18,1,29,17,0,2,0,0
3,Pazartesi,4,1,8,1,40,1,9,0,7,0,19,0,29,33,0,3,0,0
4,Perşembe,2,1,3,0,28,1,4,0,6,0,9,2,23,21,0,2,0,0


so we will decrease 3:50 and 4:50 by half an hour to reflect real-world scenarios

In [222]:
from datetime import datetime, timedelta

def selectively_shift_end_time(df, time_column='end_time'):
    """
    Shifts only '15:50' and '16:50' values in the time_column back by 30 minutes.
    Handles potential whitespace or formatting issues.
    """
    df = df.copy()

    def shift_if_needed(t):
        t = str(t).strip()  # Remove any extra whitespace
        if t in ['15:50', '16:50']:
            new_time = datetime.strptime(t, "%H:%M") - timedelta(minutes=30)
            return new_time.strftime("%H:%M")
        return t

    df[time_column] = df[time_column].apply(shift_if_needed)
    return df



In [223]:
# Apply function
shifted_df = selectively_shift_end_time(new_o_data)



In [224]:
shifted_df

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums,end_time_clean,day_clean
0,GÖRSEL İLETİŞİM TASARIMI_GSF102,GÖRSEL İLETİŞİM TASARIMI,GSF102,TEMEL TASARIM II,L-102(GİT ATÖLYE II)[60],SEÇİL KARTOPU,1,Pazartesi,09:00,14:50,41,14:50,Pazartesi
1,GÖRSEL İLETİŞİM TASARIMI_GSF106,GÖRSEL İLETİŞİM TASARIMI,GSF106,ÇİZİM II,L-101(GİT ATÖLYE I)[60],Öğr.Gör. İSMAİL FURKAN YAVUZ,1,Pazartesi,09:00,14:50,44,14:50,Pazartesi
2,GÖRSEL İLETİŞİM TASARIMI_BLG102,GÖRSEL İLETİŞİM TASARIMI,BLG102,BİLGİ TEKNOLOJİLERİ KULLANIMI,AZ 13(BİLGİSAYAR LAB.)[80],Öğr.Gör. AHMET AKSOY,1,Salı,09:00,15:20,44,15:50,Salı
3,İKTİSAT_ENG104,İKTİSAT,ENG104,ACADEMIC ENGLISH II,B382(KONFERANS SALONU)[143],Öğr.Gör. MÜGE AKGEDİK AKOĞLANOĞLU,1,Pazartesi,09:00,16:20,35,16:50,Pazartesi
4,İKTİSAT_ECON102,İKTİSAT,ECON102,INTRODUCTION TO ECONOMICS II,B338-D1(DERSLİK1)[121],KORAY GÖKSAL,1,Pazartesi,14:00,16:20,91,16:50,Pazartesi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,FELSEFE_PSİ340,FELSEFE,PSİ340,Öğrenme Psikolojisi,None,None,4,Çarşamba,09.00,11:50,31,11:50,Çarşamba
619,FELSEFE_FEL408,FELSEFE,FEL408,Siyaset Felsefesi,None,None,4,Çarşamba,09.00,11:50,36,11:50,Çarşamba
620,FELSEFE_FEL420,FELSEFE,FEL420,Felsefi Danışmanlık,None,None,4,Çarşamba,13.00,14:50,23,14:50,Çarşamba
621,FELSEFE_FEL424,FELSEFE,FEL424,İletişim Felsefesi,None,None,4,Perşembe,13.00,14:50,19,14:50,Perşembe


In [225]:
shifted_df.drop(columns=['end_time_clean', 'day_clean'], inplace=True)

In [227]:
shifted_df

,faculty_code,faculty,course_code,course_name,class,teacher,year,day,start_time,end_time,student_nums
0,GÖRSEL İLETİŞİM TASARIMI_GSF102,GÖRSEL İLETİŞİM TASARIMI,GSF102,TEMEL TASARIM II,L-102(GİT ATÖLYE II)[60],SEÇİL KARTOPU,1,Pazartesi,09:00,14:50,41
1,GÖRSEL İLETİŞİM TASARIMI_GSF106,GÖRSEL İLETİŞİM TASARIMI,GSF106,ÇİZİM II,L-101(GİT ATÖLYE I)[60],Öğr.Gör. İSMAİL FURKAN YAVUZ,1,Pazartesi,09:00,14:50,44
2,GÖRSEL İLETİŞİM TASARIMI_BLG102,GÖRSEL İLETİŞİM TASARIMI,BLG102,BİLGİ TEKNOLOJİLERİ KULLANIMI,AZ 13(BİLGİSAYAR LAB.)[80],Öğr.Gör. AHMET AKSOY,1,Salı,09:00,15:20,44
3,İKTİSAT_ENG104,İKTİSAT,ENG104,ACADEMIC ENGLISH II,B382(KONFERANS SALONU)[143],Öğr.Gör. MÜGE AKGEDİK AKOĞLANOĞLU,1,Pazartesi,09:00,16:20,35
4,İKTİSAT_ECON102,İKTİSAT,ECON102,INTRODUCTION TO ECONOMICS II,B338-D1(DERSLİK1)[121],KORAY GÖKSAL,1,Pazartesi,14:00,16:20,91
...,...,...,...,...,...,...,...,...,...,...,...
618,FELSEFE_PSİ340,FELSEFE,PSİ340,Öğrenme Psikolojisi,None,None,4,Çarşamba,09.00,11:50,31
619,FELSEFE_FEL408,FELSEFE,FEL408,Siyaset Felsefesi,None,None,4,Çarşamba,09.00,11:50,36
620,FELSEFE_FEL420,FELSEFE,FEL420,Felsefi Danışmanlık,None,None,4,Çarşamba,13.00,14:50,23
621,FELSEFE_FEL424,FELSEFE,FEL424,İletişim Felsefesi,None,None,4,Perşembe,13.00,14:50,19


In [226]:
#shifted_df.to_csv('class_model.csv', index=False)

ARCH404 	RESEARCH THESIS
ARCH402 	STUDIO VIII (FINAL PROJECT)
PUBF471 	GRADUATION PROJECT
SOS404 	BİTİRME PROJESİ II
BBY318 	BİTİRME PROJESİ I
SY406 	BİTİRME PROJESİ

FEL404 	Bitirme Tezi-II
TDE442 	ARAŞTIRMA PROJESİ II
BED402 	MEZUNİYET PROJESİ II
ÇGL410 	Mezuniyet Tez ÇalışmasıI II
 	BED404 	SEMİNER II
TAR 457 	Seminer-I

some senior projects classes we removed later